# Fall 2021 CE170A HW2: Sensor Data Analysis
Authors: Prof. Kenichi Soga, Renjie Wu, Yaobin Yang, James Wang 

version 2.0

## Install Packages

In [ ]:
import os
!pip install trusspy
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2021/master/Homeworks/HW2/data/processed_data.zip" -O processed_data.zip
!unzip processed_data

## Background

There are two ways to understand phenomena scientifically. One is through mathematical modeling and computer simulation, which is the purpose of our quiz 2 (truss finite element simulation). The other is through experiments, which are performed in this class (model bridge loading experiment). By comparing experimental results with simulation results, one may appreciate the complexity of real-world phenomena and have a deeper understanding of the power and limitations of the simulation method.

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/overview.png" alt="Node numbering" width="600" />
    
    Figure 1. Simulation Vs. Experiment
</center>

In this assignment, you are going to analyze the sensor data from the model bridge experiment and compare the result with the truss simulation performed in quiz 2. The data you are going to explore are 
* Displacement data from displacement sensors (potentiometric measuring devices),
* Tilt data from WSN tilt motes, and 
* Distributed (spatial) strain data from fiber optics sensors.



The typical workflow of data analysis is summarized in figure 2. 
<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/data_processing.png" alt="Node numbering" width="600" />
    
    Figure 2. Typical workflow of data analysis
</center>

We start with the raw data collected from the sensor systems. Then, we preprocess the raw data and store the cleaned data into a data storage system. There are various reasons for data preprocessing. Missing data handling, experiment/equipment setting adjustment, and data format conversion are common reasons (there are more, of course). 

Most data can be represented as a single spreadsheet. Such data include the displacement data and tilt data that we are going to explore in this assignment. However, in some cases, data can have complex structures; such as the distributed strain data we have in this case. The distributed strain data is collected from fiber optics sensors, which record the strain along with the fiber (spatial component) during the experiment (time component). As you all know, we deployed numerous fiber optics sensors (with different lengths) across different structure members to monitor the load responses of the model bridge. Since we have three model bridges this year, each with different sensor deployment strategies, the sensors' raw data are huge and hard to handle. For the ease of this exercise, the distributed strain data is preprocessed by us, and only some cleaned data from certain bridge members are presented here.


Having the cleaned data on our hands, we can perform various postprocessing techniques such as statistical analysis, and most importantly, visualize the data for interpretation.

The data are given in a format that follows the numbering given in the following figures.

Figure 3 shows the nodal numbering of the model bridge.
<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/figure1.png" alt="Node numbering" width="800" />
    
    Figure 3. Node numbering for the model bridge
</center>

## Data Exploration
### 1. Displacemenent data (potentiometric measuring devices) : load-displacement curves

Potentiometric measuring devices (LVDT) are used to measure the displacement of the bottom members of the model bridge. The configuration of the LVDT sensors used in the experiment are shown by the figure below. 
<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/lvdt_position.jpg" alt="Node numbering" width="800" />
    
    Figure 4. LVDT configuration for the experiment. 
</center>

#### Read the data

Reminder: the sequence of model bridges performed in the experiment is Bridge3 (groupC), Bridge2 (groupB), and Bridge1 (groupA). We will use this sequence through out the assignment

Let's read the processed data (base adjusted, unit converted, format changed) first:

In [ ]:
import numpy as np
import pandas as pd

lvdt3 = pd.read_csv('./processed_data/LVDT/lvdt_b3.csv')
lvdt2 = pd.read_csv('./processed_data/LVDT/lvdt_b2.csv')
lvdt1 = pd.read_csv('./processed_data/LVDT/lvdt_b1.csv')

lvdt1.head()

The data is represented as a table that has several columns. The first column is the load (N), and the rest of the columns are the corresponding displacement readings of the LVDT sensors, configured by Figure 4.

#### The loading processes

Before proceeding to analyze the LVDT data, let's take a look at the loading process first. 

In [ ]:
load3 = lvdt3['Load (N)'].to_numpy()
load2 = lvdt2['Load (N)'].to_numpy()
load1 = lvdt1['Load (N)'].to_numpy()

The plotting function is provided in this assignment. In the cell below, we give an example on how to use the function to plot the load. Note that the plot_lines function is capable of plotting multiple lines.

In [ ]:
import matplotlib.pyplot as plt

def plot_lines(xs,ys,xy_labels,line_lables):
    """ A multi-line plotter
     Args:
      xs: list of x values, i.e. [x1,x2,x3...]
      ys: list of y values, i.e. [y1,y2,y3...]
      xy_labels: labels for the figure, in the form of (x_label, y_label)
      line_lables: labels for plotted lines, must match to the order of yvalues in y

    Returns:
      ax: matplotlib axes 
    """
    assert len(xs)==len(ys), "number of x must equal to number of y"
    assert len(ys)==len(line_lables),  "ys and line_lables must be list with same shape, i.e. [y1],[y1_label]"
    
    fig = plt.figure()
    ax=fig.add_axes([0,0,1,1])
    
    for x,y in zip(xs,ys):
        l = ax.plot(x,y)
    ax.set_xlabel(xy_labels[0])
    ax.set_ylabel(xy_labels[1])
    ax.legend(labels = line_lables)
    return ax

In [ ]:
# x-axis is number of time steps in this case 
x3 = list(range(len(load3)))
x2 = list(range(len(load2)))
x1 = list(range(len(load1)))

plot_lines([x3,x2,x1],[load3,load2,load1],['Time Steps','Load(N)'],['load3','load2','load1'])

<font color='red'>Question: </font> : What is the maximum load for each experiment? What are the corresponding time steps for them? Do the measurements match your on-site observation? 

<font color='red'>Answer: </font> 

#### Bottom Members Movements

Now let's take a look at how the bottom members respond to the load. To see the expansion of the bottom members, plot load Vs. displacement for LVDT 2 3 6 7. Take the relative displacment of two LVDTs to find the compression or extension behavior of the bottom members with increasing load.

In [ ]:
# Model bridge3 
d2 = lvdt3['d2 (m)'].to_numpy()
d3 = lvdt3['d3 (m)'].to_numpy()
d6 = lvdt3['d6 (m)'].to_numpy()
d7 = lvdt3['d7 (m)'].to_numpy()


# Only care about the loading process 
mask = np.array(x3)< 5700 #use the time step threshold you answered before 

plot_lines([d2[mask],d3[mask],d6[mask],d7[mask]],[load3[mask]]*4,
           ['displacement(m)','load(N)'],['d2','d3','d6','d7'])

In [ ]:
# It is the relative displacement we want to measure, take the difference to find it out 
plot_lines([d2[mask]-d7[mask],d3[mask]-d6[mask],],[load3[mask]]*2,
           ['displacement(m)','load(N)'],['d2-7','d3-6'])

<font color='red'>Question: </font> : Try to describe the behavior of the bottom members (compresison or extension) using the plotted figure. 

<font color='red'>Answer: </font> 

In [ ]:
# Model bridge2
d2 = lvdt2['d2 (m)'].to_numpy()
d3 = lvdt2['d3 (m)'].to_numpy()
d6 = lvdt2['d6 (m)'].to_numpy()
d7 = lvdt2['d7 (m)'].to_numpy()

# Only care about the loading process 
mask = np.array(x2)<  #TODO use the time step threshold you answered before 

plot_lines([d2[mask],d3[mask],d6[mask],d7[mask]],[load2[mask]]*4,
           ['displacement(m)','load(N)'],['d2','d3','d6','d7'])

In [ ]:
# TODO plot the relative displacement for the two members

<font color='red'>Question: </font> : Try to describe the behavior of the bottom members (compression or extension) using the plotted figure. 

<font color='red'>Answer: </font> 

In [ ]:
# Model bridge1
d2 = lvdt1['d2 (m)'].to_numpy()
d3 = lvdt1['d3 (m)'].to_numpy()
d6 = lvdt1['d6 (m)'].to_numpy()
d7 = lvdt1['d7 (m)'].to_numpy()


# Only care about the loading process 
mask = np.array(x1)<  #TODO use the time step threshold you answered before 

plot_lines([d2[mask],d3[mask],d6[mask],d7[mask]],[load1[mask]]*4,
           ['displacement(m)','load(N)'],['d2','d3','d6','d7'])

In [ ]:
# TODO plot the relative displacement for the two members

<font color='red'>Question: </font> : Try to describe the behavior0 of the bottom members (compression or extension) using the plotted figure. 

<font color='red'>Answer: </font> 

<font color='red'>Question: </font> : Discuss the similarities and differences of the bottom member behavior for the three cases. Do the experiment measurements match your simulation prediction in quiz 2? 

<font color='red'>Answer: </font> 

### 2. Tilt data (WSN tilt motes) : load-tilt curves

Lets explore the tilt data from WSN tilt motes. Figure 5 shows the location of the WSN tilt motes and their numbering. The direction of angle is given for both longitudinal direction (L) and transverse direction (T).

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/tilt_defination.jpg" alt="Node numbering" width="600" />
    
    Figure 5. The locations of WSN tilt motes.
</center>

In [ ]:
# Load Logitudinal Tilt (right-left direction, "L direction" in Figure 5
bridge3 = pd.read_excel(open('./processed_data/Tilt/tilt_data.xlsx', 'rb'),
              sheet_name='test_1')  
bridge2 = pd.read_excel(open('./processed_data/Tilt/tilt_data.xlsx', 'rb'),
              sheet_name='test_2')  
bridge1 = pd.read_excel(open('./processed_data/Tilt/tilt_data.xlsx', 'rb'),
              sheet_name='test_3')  


In [ ]:
bridge1.head()

The data contains the tilt measurements (in degree) from each sensor with the corresponding load levels. Obviously, the entire dataset contains rich information about the structure state of the model bridge during the loading process. However, since this is already a long assignment, we will only focus on the tilt measurements from the top sensor, which gives us the general information about the rotation degree of the model bridge.

In [ ]:
# Lets start with bridge 3 
load = bridge3['Loading, lbs'].to_numpy() *4.45 # to newton 
long = bridge3['Top-L, deg'].to_numpy()
tran = bridge3['Top-T, deg'].to_numpy()

Plot the longitudinal tilt-load curve (load on x-axis, tilt on y-axis) of the top using the tilt WNS mote data (Tilt 1) BEFORE the failure.

In [ ]:
plot_lines([load]*2,[long,tran],
           ['load(N)','tilting degree'],['Top (longitudinal)','Top (transverse)'])

<font color='red'>Question: </font> : Describe the movement of the model bridge during the loading process. Does this matches your observation? 

Note: The figure below shows the final stage of the model bridge. 

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/b3.png" alt="Node numbering" width="400" />
    
    Model bridge final state (bridge 3)
</center>

<font color='red'>Answer: </font> 

In [ ]:
# bridge 2
load = bridge2['Loading, lbs'].to_numpy() *4.45 # to newton 
long = bridge2['Top-L, deg'].to_numpy()
tran = bridge2['Top-T, deg'].to_numpy()

plot_lines([load]*2,[long,tran],
           ['load(N)','tilting degree'],['Top (longitudinal)','Top (transverse)'])

<font color='red'>Question: </font> : Describe the movement of the model bridge during the loading process. Does this matches your observation? 

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/b2.png" alt="Node numbering" width="400" />
    
    Model bridge final state (bridge 2)
</center>

<font color='red'>Answer: </font> 

In [ ]:
# bridge 2
load = bridge1['Loading, lbs'].to_numpy() *4.45 # to newton 
long = bridge1['Top-L, deg'].to_numpy()
tran = bridge1['Top-T, deg'].to_numpy()

plot_lines([load[:]]*2,[long[:],tran[:]],
           ['load(N)','tilting degree'],['Top (longitudinal)','Top (transverse)'])

<font color='red'>Question: </font> : Describe the movement of the model bridge during the loading process. Does this matches your observation? 

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/b1.png" alt="Node numbering" width="400" />
    
    Model bridge final state (bridge 1)
</center>

<font color='red'>Answer: </font> 

### 3. Strain data (fiber optics): load-strain curves
Finally, let's explore the most interesting data, the distributed strain data collected from the fiber optics sensors. The model bridge memberes are divided into 4 planes: Bottom, Left-diagonal, Right-diagonal, and Middle plane. See figure 7. 

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/figure1.png" alt="Node numbering" width="800" />
    
    Figure 7. Node numbering for the model bridge
</center>

As discussed earlier, the raw data collected by the fiber optics sensors are spatially distributed. Specifically, the data contains spatial strain information at each sample time for each fiber. The general preprocessing process is listed as the following:
* Cleaning: Convert data format, remove headers, convert units, remove meaningless items... 
* Time synchronization: all the collected data is time-stamped. Unfortunately, the timing systems for different measurement equipment might be slightly different. For instance, the clock in the fiber optics analyzer is 2 minutes and 19 seconds faster than the one measuring the loading process. Synchronizing times across all equipment make the comparison/plotting of different measurements possible. 
* Resampling: sometimes, the sampling rates across different equipment are different. To compare different measurements from different machines (i.e., load vs. strain), one needs to resample the collected data to match the time stamp.
* Normalization: some fibers are pre tensioned, meaning the starting measurement may not be 0. Doing normalization for the data makes the measurement (strain) comparable across different bridges. 
 


 <font color='red'>Note: the data you see in this section is far from perfect (the fibers are attached by you!). If some data does not make any sense to you, think about the potential causes for that. Try your best to discuss your findings.  </font>

The preprocessed data for model bridge 3 is shown by the cell below (measurements of one fiber only).

In [ ]:
sample_data = pd.read_csv('./processed_data/Fiber/example_strain_b3c3.csv')
sample_data.head()

#### Spatial Data Analaysis

The first row of the data provides the spatial location for each strain measurement points alone the fiber. The rest of the rows are the strain measurements across the fiber (2.6 mm resolution) for every sample time. To access the strain measurement for certain truss members, one need to know the corresponding spatial locations of the member on the fiber. Luckily, our kind GSI did indexing (truss member end points to fiber locations) for all the truss members for all three bridges, so we know how to access the member from the data. For example, member JC in bridge3 corresponding to the range 9.123 to 9.539m. With some codes, we can access all the strain measurement for member JC easily:  

In [ ]:
def get_position(ends,pos):
    start, end = ends 
    cols = []
    for i, p in enumerate(pos):
        if start<=p<=end:
            cols.append(i)
    return cols
pos = list(sample_data.iloc[0])[2:]
cols = list(map(str,get_position([9.123,9.539],pos)))
JC_strain = sample_data[cols]


Let's plot the strain response of it under intense load.

In [ ]:
# plot at one time step
plot_lines([JC_strain.iloc[0]],[JC_strain.iloc[100]],
           ['location(m)','micro strain'],['Strain for JC'])

<font color='red'>Question: </font> : Note that the two end points of the member have very small strain. Discuss the potential reasons for it (think about how you attached it to the bridge, what did you do around the joints?) 

<font color='red'>Answer: </font>

Let's try to plot a longer member FC. 

In [ ]:
pos = list(sample_data.iloc[0])[2:]
cols = list(map(str,get_position([13.135,14.401],pos)))
FC_strain = sample_data[cols]
# plot at one time step
plot_lines([FC_strain.iloc[0]],[FC_strain.iloc[100]],
           ['location(m)','micro strain'],['Strain for FC'])


<font color='red'>Question: </font> : Can you interpret the deformation of FC from the data? Discuss the differences between the measurements on JC and FC. 

**Hint: Fibers are attached on DIFFERENT sides of the angle bar for FC and JC. Also, FC is two times longer than JC. What tend to happen on long truss elements during the loading process?**

<font color='red'>Answer: </font>

#### Time (load) Data Analysis



Since different groups wired fiber cables in different ways and analyzing both spatial and temporal data requires skills beyond the boundary of this class, simplified strain data is used in the later part of the assignment. Certain truss members are extracted from the data for all bridges, and only the average (spatial averaged) strain for each truss member is provided along with the corresponding load levels. 

In [ ]:
# load the data 
bridge1 = pd.read_csv('./processed_data/Fiber/fo_bridge1.csv')
bridge2 = pd.read_csv('./processed_data/Fiber/fo_bridge2.csv')
bridge3 = pd.read_csv('./processed_data/Fiber/fo_bridge3.csv')

In [ ]:
bridge3.head()

Let's start with Bridge 3 (first bridge).

In [ ]:
def subplot_lines(xs,ys,xy_labels,line_lables):
    """Subplot function 
     Args:
      xs: list of x values, i.e. [x1,x2,x3...]
      ys: list of y values, i.e. [y1,y2,y3...]
      xy_labels: labels for the figure, in the form of (x_label, y_label)
      line_lables: labels for plotted lines, must match to the order of yvalues in y
    Returns:
        None
    """
    assert len(ys)==len(line_lables),  "ys and line_lables must be list with same shape, i.e. [y1],[y1_label]"
    
    fig, axes = plt.subplots(1, len(ys),figsize=(18,9))
    
    for ax, x, y,label in zip(axes,xs,ys,line_lables): 
        l = ax.plot(x,y,label =label)
        ax.legend()
        
        
    # add a big axes, hide frame
    ax = fig.add_subplot(111, frameon=False)
    # hide tick and tick label of the big axes
    ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    ax.grid(False)
    ax.set_xlabel(xy_labels[0])
    ax.set_ylabel(xy_labels[1])
    ax.legend(labels = line_lables)






In [ ]:
bridge3 = bridge3.iloc[:1000] # only the loading process 

In [ ]:
# Diagonal Members 
member_names = ['AG','IB','DH','JC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge3['load (N)'])/1000)
    strains.append(np.array(bridge3[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? 

Note: the wiggles are artifacts due to imperfect time matching between the fiber optics sensor analyzer and the load recording machine. 

<font color='red'>Answer: </font>

In [ ]:
# Bottom Members 
member_names = ['AM','MB','DN','NC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge3['load (N)'])/1000)
    strains.append(np.array(bridge3[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? Does this matches the LVDT measurements? 

Note: the wiggles are artifacts due to imperfect time matching between the fiber optics sensor analyzer and the load recording machine. 

<font color='red'>Answer: </font>

Bridge 2. Unfortunately, we only have data on one side of bridge2 due to the connection issues we had during the experiment.

In [ ]:
bridge2 = bridge2.iloc[:600] # only the loading process 

In [ ]:
# Diagonal Members 
member_names = ['DH','JC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge2['load (N)'])/1000)
    strains.append(np.array(bridge2[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? 

<font color='red'>Answer: </font>

In [ ]:
# Bottom Members 
member_names = ['DN','NC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge2['load (N)'])/1000)
    strains.append(np.array(bridge2[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? Does this matches the LVDT measurements? 

Note: the wiggles are artifacts due to imperfect time matching between the fiber optics sensor analyzer and the load recording machine. 

<font color='red'>Answer: </font>

Bridge1 

In [ ]:
bridge1 = bridge1.iloc[:150] # only the loading process 

In [ ]:
# Diagonal Members 
member_names = ['AG','IB','DH','JC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge1['load (N)'])/1000)
    strains.append(np.array(bridge1[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? 

<font color='red'>Answer: </font>

In [ ]:
# Bottom Members 
member_names = ['AM','MB','DN','NC']
loads = []
strains = []
for member_name in member_names:
    loads.append(np.array(bridge1['load (N)'])/1000)
    strains.append(np.array(bridge1[member_name]))
subplot_lines(strains,loads,['micro strain','Load (kN)'],member_names)

<font color='red'>Question: </font> : Describe any similarities and differences between the members. Does it match your prediction? Does this matches the LVDT measurements? 

<font color='red'>Answer: </font>

## Discussion


### Material stiffness (Young's modulus) revision using load-strain curves

Use the stress-strain plots above to revise the stiffness of the material in your FE model. Are your estimations the same across different bridges?  

Hint1: "load" from data is the system (total) load, go back to quiz 2 to find the equation to convert total load to the load felt by each member. Again, pay attention to number of angle bars for each member. Your answer should have the same magnitude to the value you have in quiz 2.

Hint2: choose a good compression data on the diagnonal members (i.e. IB in bridge 1)! If you dont know how to handle the data, reading the values from the graph is also acceptable.

<font color='red'>Answer: </font>

Describe the deformation mechanisms of the bridge as a whole under at different loading stages. 

<font color='red'>Answer: </font> 



Discuss the validity of the assumptions that we made in the analysis given in quiz 2.


<font color='red'>Answer: </font> 


If you want to improve the simulation analysis, what would you do? 

<font color='red'>Answer: </font> 


If you are allowed to add more sensors to the model bridge, what and where would you add? 

<font color='red'>Answer: </font> 
